In [1]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
import tensorflow as tf
from tensorflow import keras

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam

In [56]:
import urllib
response = urllib.request.urlopen("http://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt")
sonnet = response.read().decode("utf-8")
print(sonnet)

FROM fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou, contracted to thine own bright eyes,
Feed'st thy light'st flame with self-substantial fuel,
Making a famine where abundance lies,
Thyself thy foe, to thy sweet self too cruel.
Thou that art now the world's fresh ornament
And only herald to the gaudy spring,
Within thine own bud buriest thy content
And, tender churl, makest waste in niggarding.
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.
When forty winters shall beseige thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery, so gazed on now,
Will be a tatter'd weed, of small worth held:
Then being ask'd where all thy beauty lies,
Where all the treasure of thy lusty days,
To say, within thine own deep-sunken eyes,
Were an all-eating shame and thriftless praise.
How much more praise deserved thy b

In [57]:
corpus = sonnet.lower().split("\n")

In [58]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
# One word for unknown
total_words = len(tokenizer.word_index) + 1

In [59]:
print(total_words)

3211


In [60]:
word_index = tokenizer.word_index

In [61]:
print(word_index)

{'and': 1, 'the': 2, 'to': 3, 'of': 4, 'my': 5, 'i': 6, 'in': 7, 'that': 8, 'thy': 9, 'thou': 10, 'with': 11, 'for': 12, 'is': 13, 'love': 14, 'not': 15, 'but': 16, 'a': 17, 'me': 18, 'thee': 19, 'so': 20, 'be': 21, 'as': 22, 'all': 23, 'you': 24, 'his': 25, 'which': 26, 'when': 27, 'it': 28, 'this': 29, 'by': 30, 'your': 31, 'doth': 32, 'do': 33, 'from': 34, 'on': 35, 'or': 36, 'no': 37, 'then': 38, 'have': 39, 'what': 40, 'are': 41, 'if': 42, 'more': 43, 'mine': 44, 'their': 45, 'shall': 46, 'sweet': 47, 'time': 48, 'will': 49, 'they': 50, 'beauty': 51, 'nor': 52, 'eyes': 53, 'art': 54, 'her': 55, 'heart': 56, 'yet': 57, 'o': 58, 'than': 59, 'can': 60, 'should': 61, 'thine': 62, 'now': 63, 'where': 64, 'make': 65, 'one': 66, 'hath': 67, 'he': 68, 'fair': 69, 'still': 70, 'how': 71, 'eye': 72, 'him': 73, 'like': 74, 'true': 75, 'see': 76, 'am': 77, 'she': 78, 'those': 79, 'though': 80, 'being': 81, 'some': 82, 'every': 83, 'such': 84, 'own': 85, 'were': 86, 'dost': 87, 'who': 88, 'liv

In [62]:
# input_sequences = []
# for line in corpus:
#     token_list = tokenizer.texts_to_sequences([line])[0]
#     for i in range(1, len(token_list)):
#         n_gram_sequence = token_list[:i+1]
#         input_sequences.append(n_gram_sequence)
#         print(input_sequences)
#     break

[[34, 417]]
[[34, 417], [34, 417, 877]]
[[34, 417], [34, 417, 877], [34, 417, 877, 166]]
[[34, 417], [34, 417, 877], [34, 417, 877, 166], [34, 417, 877, 166, 213]]
[[34, 417], [34, 417, 877], [34, 417, 877, 166], [34, 417, 877, 166, 213], [34, 417, 877, 166, 213, 517]]


In [63]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [64]:
max_sequence_len = max([len(x) for x in input_sequences])
print(max_sequence_len)

11


In [65]:
# pre pad with zero which makes it easier to extract the labels
input_sequences = np.array(pad_sequences(input_sequences,
                                         maxlen = max_sequence_len,
                                         padding = 'pre')
                          )

In [66]:
input_sequences[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,  34, 417],
       [  0,   0,   0,   0,   0,   0,   0,   0,  34, 417, 877],
       [  0,   0,   0,   0,   0,   0,   0,  34, 417, 877, 166],
       [  0,   0,   0,   0,   0,   0,  34, 417, 877, 166, 213],
       [  0,   0,   0,   0,   0,  34, 417, 877, 166, 213, 517]])

In [67]:
X = input_sequences[:,:-1]
labels = input_sequences[:,-1]

In [68]:
# converts labels to one hot encoding vector
y = tf.keras.utils.to_categorical(labels, num_classes = total_words)

In [69]:
print(tokenizer.word_index['from'])
print(tokenizer.word_index['fairest'])
print(tokenizer.word_index['creatures'])
print(tokenizer.word_index['we'])
print(tokenizer.word_index['desire'])
print(tokenizer.word_index['increase'])

34
417
877
166
213
517


In [70]:
print(X[4])
print(len(y[4]))
# print(y[4])

[  0   0   0   0   0  34 417 877 166 213]
3211


In [71]:
# length is reduced by one as last sequence is moved to label
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(total_words, activation='softmax'))

In [72]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [73]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 64)            205504    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 40)                13600     
_________________________________________________________________
dense_2 (Dense)              (None, 3211)              131651    
Total params: 350,755
Trainable params: 350,755
Non-trainable params: 0
_________________________________________________________________
None


In [74]:
history = model.fit(X, y, epochs=150, verbose=1)

Train on 15462 samples
Epoch 1/150
15462/15462 [==============================] - 18s 1ms/sample - loss: 6.9273 - accuracy: 0.0208s - loss: 6.9294 - ac
Epoch 2/150
15462/15462 [==============================] - 11s 698us/sample - loss: 6.5013 - accuracy: 0.0237
Epoch 3/150
15462/15462 [==============================] - 11s 722us/sample - loss: 6.4161 - accuracy: 0.0286
Epoch 4/150
15462/15462 [==============================] - 12s 779us/sample - loss: 6.3398 - accuracy: 0.0339
Epoch 5/150
15462/15462 [==============================] - 13s 828us/sample - loss: 6.2617 - accuracy: 0.0382
Epoch 6/150
15462/15462 [==============================] - 12s 759us/sample - loss: 6.1756 - accuracy: 0.0412- loss: 6.1712 
Epoch 7/150
15462/15462 [==============================] - 13s 823us/sample - loss: 6.0897 - accuracy: 0.0417
Epoch 8/150
15462/15462 [==============================] - 13s 837us/sample - loss: 5.9983 - accuracy: 0.0461- loss: 6.0036 - accuracy: 
Epoch 9/150
15462/15462 [===========

15462/15462 [==============================] - 7s 445us/sample - loss: 1.5907 - accuracy: 0.6555 - loss: 1.5909 - accuracy: 0.65
Epoch 143/150
15462/15462 [==============================] - 7s 456us/sample - loss: 1.5719 - accuracy: 0.6623 - loss: 1.5691 - accura - ETA: 0s - loss: 1 - ETA: 0s - loss: 1.5709 - accuracy: 0.66
Epoch 144/150
15462/15462 [==============================] - 7s 473us/sample - loss: 1.5713 - accuracy: 0.6592
Epoch 145/150
15462/15462 [==============================] - 7s 477us/sample - loss: 1.5620 - accuracy: 0.6588
Epoch 146/150
15462/15462 [==============================] - 7s 472us/sample - loss: 1.5549 - accuracy: 0.6641
Epoch 147/150
15462/15462 [==============================] - 7s 468us/sample - loss: 1.5401 - accuracy: 0.6655 - loss: 1.5
Epoch 148/150
15462/15462 [==============================] - 7s 459us/sample - loss: 1.5244 - accuracy: 0.6733
Epoch 149/150
15462/15462 [==============================] - 7s 481us/sample - loss: 1.5141 - accuracy: 0.6

In [54]:
y.shape

(5, 3211)

In [77]:
from tensorflow.keras import regularizers
# Bigger model
model = Sequential()
model.add(Embedding(total_words, 64, input_length= max_sequence_len - 1))
model.add(Bidirectional(LSTM(100,return_sequences = True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(25)))
model.add(Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(l=0.1)))
model.add(Dense(total_words, activation = 'softmax'))

In [78]:
adam = Adam(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 10, 64)            205504    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 10, 200)           132000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 200)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 50)                45200     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               6528      
_________________________________________________________________
dense_4 (Dense)              (None, 3211)              414219    
Total params: 803,451
Trainable params: 803,451
Non-trainable params: 0
________________________________________________

In [79]:
history = model.fit(X, y, epochs=150, verbose=1)

Train on 15462 samples
Epoch 1/150
15462/15462 [==============================] - 28s 2ms/sample - loss: 6.9481 - accuracy: 0.0218
Epoch 2/150
15462/15462 [==============================] - 22s 1ms/sample - loss: 6.5317 - accuracy: 0.0234
Epoch 3/150
15462/15462 [==============================] - 22s 1ms/sample - loss: 6.4581 - accuracy: 0.0252
Epoch 4/150
15462/15462 [==============================] - 21s 1ms/sample - loss: 6.4222 - accuracy: 0.0299
Epoch 5/150
15462/15462 [==============================] - 28s 2ms/sample - loss: 6.4057 - accuracy: 0.0309
Epoch 6/150
15462/15462 [==============================] - 23s 1ms/sample - loss: 6.3822 - accuracy: 0.0311
Epoch 7/150
15462/15462 [==============================] - 22s 1ms/sample - loss: 6.3529 - accuracy: 0.0315
Epoch 8/150
15462/15462 [==============================] - 22s 1ms/sample - loss: 6.3242 - accuracy: 0.0318
Epoch 9/150
15462/15462 [==============================] - 20s 1ms/sample - loss: 6.3051 - accuracy: 0.0321
Epoch

15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6922 - accuracy: 0.0608
Epoch 77/150
15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6876 - accuracy: 0.0633
Epoch 78/150
15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6999 - accuracy: 0.0625
Epoch 79/150
15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6913 - accuracy: 0.0625
Epoch 80/150
15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6884 - accuracy: 0.0616
Epoch 81/150
15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6881 - accuracy: 0.0612
Epoch 82/150
15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6752 - accuracy: 0.0625
Epoch 83/150
15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6751 - accuracy: 0.0660
Epoch 84/150
15462/15462 [==============================] - 16s 1ms/sample - loss: 5.6665 - accuracy: 0.0629
Epoch 85/150
15462/15462 [======